In [1]:
import numpy as np
import math
import xlwings as xw

In [2]:
#Variables
S = 100 #Stock price at t=0
r = 0.05  #Risk free rate
dt = .01  #Timestep size
T = 1  #time to expiry in years
sigma = .250
E = 100  #Strike Price

length = T/dt
u = math.exp( sigma * math.sqrt(dt))
d = math.exp(- sigma * math.sqrt(dt))
a = math.exp( r * dt)
p = (a - d) / (u - d)

u,d,a,p

(1.0253151205244289, 0.9753099120283326, 1.0005001250208359, 0.503751784066049)

In [3]:
#Create Empty Numpy Arrays to hold the data
step_matrix = np.zeros((int(length), int(length+1)))
call_option_value_matrix = np.zeros((int(length+1), int(length+1)))
put_option_value_matrix = np.zeros((int(length+1), int(length+1)))

stock_value_matrix = np.zeros((int(length+1), int(length+1)))
#Set initial value for the stock
stock_value_matrix[0,0] = S

In [4]:
#Create binomial tree of the pricing of the underlying
for x in range(1,int(length+1)):
    stock_value_matrix[x,0] = stock_value_matrix[x-1,0]*(d)
    for y in range(1,int(length+1)):
        stock_value_matrix[x,y] = stock_value_matrix[x-1,y-1]*(u)
#xw.view(value_matrix)
#stock_value_matrix

In [5]:
#Create Option Valuation Tree Call Option
for x in range(1, int(length+1)):
    for y in range(0, int(length+1)):
        if stock_value_matrix[-x , y] - E > 0:
            call_option_value_matrix[-x, y] = stock_value_matrix[-x , y] - E
        else:
            call_option_value_matrix[-x , y] = 0
#xw.view(call_option_value_matrix)
            
#Create Option Valuation Tree Put Option
for x in range(1, int(length+1)):
    for y in range(0, int(length+1)):
        if E - stock_value_matrix[-x , y] > 0:
            put_option_value_matrix[-x, y] = E - stock_value_matrix[-x , y]
        else:
            put_option_value_matrix[-x , y] = 0

In [6]:
call_payoff_matrix = np.zeros((int(length+1), int(length+1)))
call_payoff_matrix[-1] = call_option_value_matrix[-1]
count = 0
for x in range(1,int(length+1)):
    count += 1
    for y in range(count, int(length+1)):
        call_payoff_matrix[-x-1,-y-1] = math.exp(-r*dt)*((p*call_payoff_matrix[-x,-y])+((1-p)*call_payoff_matrix[-x,-y-1]))
        if call_payoff_matrix[-x-1,-y-1] < call_option_value_matrix[-x-1,-y-1]:
            call_payoff_matrix[-x-1,-y-1] = call_option_value_matrix[-x-1,-y-1]
        
put_payoff_matrix = np.zeros((int(length+1), int(length+1)))
put_payoff_matrix[-1] = put_option_value_matrix[-1]
count = 0
for x in range(1,int(length+1)):
    count += 1
    for y in range(count, int(length+1)):
        put_payoff_matrix[-x-1,-y-1] = math.exp(-r*dt)*((p*put_payoff_matrix[-x,-y])+((1-p)*put_payoff_matrix[-x,-y-1]))
        if put_payoff_matrix[-x-1,-y-1] < put_option_value_matrix[-x-1,-y-1]:
            put_payoff_matrix[-x-1,-y-1] = put_option_value_matrix[-x-1,-y-1]
#xw.view(put_payoff_matrix)
xw.view(call_payoff_matrix)
call_value = call_payoff_matrix[0,0]
put_value = put_payoff_matrix[0,0]

print("Value of Call Option Binomial = %.4f " %call_value)
print("Value of Put Option Binomial =  %.4f " %put_value)

Value of Call Option Binomial = 12.3113 
Value of Put Option Binomial =  7.9636 
